In [ ]:
# 한국투자증권 API 를 쉽게 사용하기 위한 라이브러리 입니다.
# !pip install mojito2

In [16]:
import mojito
from env import APP_KEY, SECRET_KEY, ACCOUNT

broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=SECRET_KEY,
    acc_no=ACCOUNT,
    mock=True
)

print(broker)

In [18]:
# 가격 조회

samsung = broker.fetch_price("005930")
print("Open :  ", samsung['output']['stck_oprc'])   # 시가
print("High : ", samsung['output']['stck_hgpr'])    # 고가
print("Low  : ", samsung['output']['stck_lwpr'])     # 저가
print("Close: ", samsung['output']['stck_prpr'])    # 종가

Open:   70200
High :  70300
Low  :  69600
Close:  70300


In [24]:
# 잔고 조회
balance = broker.fetch_balance()['output2'][0]

print("예수금 :", balance['dnca_tot_amt'])
print("총 평가 금액 :", balance['tot_evlu_amt'])

예수금 : 500000000
총 평가 금액 : 500000000


### 주문하는 (가격 * 수량)은 반드시 예수금보다 많아야 한다.

### 주문은 반드시 호가 단위로 주문해야 한다. 그렇지 않으면 주문이 성사되지 않는다.

| 기준가 | 호가단위 |
|------|------|
|   ~2,000원 미만   |   1원   |
|   2,000원 이상~5,000원 미만   |   5원   |
|   5,000원 이상~20,000원 미만   |   10원   |
|   20,000원 이상~50,000원 미만   |   50원   |
|   50,000원 이상~200,000원 미만   |   100원   |
|   200,000원 이상~500,000원 미만   |   500원   |
|   500,000원 이상   |   1000원   |
|      |      |


In [25]:
# 지정가 매수
# 지정가 매수는 가격을 입력해야 하며 주문이 바로 성사되지 않을 수 있다. return 되는 "output" 은 주문을 다시 취소하는데 사용된다.

broker.create_limit_buy_order(
    symbol="005930",
    price=65000,
    quantity=1
)

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}


In [ ]:
# 시장가 매수
# 시장가 매수는 지정가 매수와 달리 가격을 입력하지 않는다. 
broker.create_market_buy_order(
    symbol="005930",
    quantity=10
)

In [26]:
# 지정가 매도
broker.create_limit_sell_order(
    symbol="005930",
    price=67000,
    quantity=1
)

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}

In [ ]:
# 시장가 매도
broker.create_market_sell_order(
    symbol="005930",
    quantity=1
)

### 주문 성공시 리턴 값

```python
{'msg1': '주문 전송 완료 되었습니다.',
 'msg_cd': 'APBK0013',
 'output': {'KRX_FWDG_ORD_ORGNO': '91252',
            'ODNO': '0000116058',
            'ORD_TMD': '145543'},
 'rt_cd': '0'}
```

KRX_FWDG_ORD_ORGNO 은 "한국거래소 전송주문 조직번호" 을 의미합니다.<br>
ODNO 은 "원주문번호" 를 의미합니다.<br>
위 2개의 값으로 주문 취소를 할 수 있습니다.


In [27]:
# 주문 전체 취소

# org_no = KRX_FWDG_ORD_ORGNO
# order_no = ODNO
broker.cancel_order(
    org_no="91252",
    order_no="0000119206",
    quantity=4,  # 잔량전부 취소시 원주문 수량과 일치해야함
    total=True   # 잔량전부를 의미
)

{'rt_cd': '1', 'msg_cd': '40320000', 'msg1': '모의투자 원주문번호가 존재하지 않습니다.'}

In [28]:
# 주문 일부 취소
broker.cancel_order(
    org_no="91252",
    order_no="0000120154",
    quantity=2,     # 취소하고자하는 수량
    total=False     # 잔량일부
)

{'rt_cd': '1', 'msg_cd': '40320000', 'msg1': '모의투자 원주문번호가 존재하지 않습니다.'}

In [ ]:
# 주문 전체 수정
# order_type 은 뭘까
broker.modify_order(
    org_no="91252",
    order_no="0000138450",
    order_type="00",
    price=60000,
    quantity=4,
    total=True  
)

In [ ]:
# 주문 일부 수정
broker.modify_order(
    org_no="91252",
    order_no="0000143877",
    order_type="00",
    price=60000,
    quantity=2,
    total=False
)

In [34]:
# 100일간 일봉 데이터

import pandas as pd
daily = broker.fetch_ohlcv(
    symbol="005930",
    timeframe='D', # D = 일봉, W = 주봉, Y = 년봉
    adj_price=True
)

df = pd.DataFrame(daily['output2'])
dt = pd.to_datetime(df['stck_bsop_date'], format="%Y%m%d")
df.set_index(dt, inplace=True)
df = df[['stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr']]
df.columns = ['open', 'high', 'low', 'close']
df.index.name = "date"
df

,open,high,low,close
date,,,,
2023-09-08,55500,73600,54500,70300
2022-12-29,79400,79800,51800,55300
2021-12-30,81000,96800,68300,78300
2020-12-30,55500,81300,42300,81000
2019-12-30,39400,57300,36850,55800
2018-12-28,51379,54139,38100,38700
2017-12-28,35979,57519,35539,50959
2016-12-29,25199,36599,21759,36039
2015-12-30,26799,30199,20659,25199
